In [147]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import scanpy.api as sc
import pandas as pd
import h5py
import os
import time
import itertools

from util import *
# from data_loader import * 
import scdd as sd
import data_loader as dl
import dist_deconv_1d as dd1d

%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [86]:
theta = 2
r = 3
Nr = 4

In [87]:
## moments 
M = np.ones([4])
for i in range(4):
    M[i] = theta**(i+1) * np.math.factorial(r+i) / np.math.factorial(r-1)
var_ = r*theta**2
var_x2_ = theta**4 * r * (r+1) * (4*r+6)
print('check var_', var_ - (M[1]-M[0]**2))
print('check var_x2_', var_x2_ - (M[3]-M[1]**2))

check var_ 0.0
check var_x2_ 0.0


In [116]:
## check Sigma 
np.random.seed(0)
n_sample = 100000
x_sample = np.random.gamma(r,theta,n_sample)
size_factor = np.random.randn(n_sample)*0.1+1
y_sample = np.random.poisson(Nr*size_factor*x_sample)
M_sample = np.zeros([n_sample,2])
M_sample[:,0] = y_sample/Nr/size_factor
M_sample[:,1] = (y_sample**2-y_sample)/Nr**2/size_factor**2
empirical_Sigma = np.cov(M_sample.T)

## theoretical value 
theoreical_Sigma = np.zeros([2,2])
c_gamma_1 = np.mean(1/size_factor)
c_gamma_2 = np.mean(1/size_factor**2)
theoreical_Sigma[0,0] = r*theta**2 + c_gamma_1/Nr*r*theta
theoreical_Sigma[0,1] = 2*r*(r+1)*theta**3 + 2*c_gamma_1/Nr*r*(r+1)*theta**2
theoreical_Sigma[1,0] = 2*r*(r+1)*theta**3 + 2*c_gamma_1/Nr*r*(r+1)*theta**2
theoreical_Sigma[1,1] = 2*(2*r+3)*(r+1)*r*theta**4 \
                        + 4*c_gamma_1/Nr*r*(r+1)*(r+2)*theta**3\
                        + 2*c_gamma_2/Nr**2*r*(r+1)*theta**2

print('check Sigma')
print( (empirical_Sigma - theoreical_Sigma) / empirical_Sigma)

check Sigma
[[-0.00716795 -0.00486488]
 [-0.00486488 -0.00369305]]


In [98]:
## check h 
def h(x,y):
    return np.array([y/x-x, x**2/(y-x**2)])

print('check h:', theta-h(M[0],M[1])[0], r-h(M[0],M[1])[1])

## check jacobian 
numerical_J = np.zeros([2,2])
step = 1e-8

numerical_J[:,0] = (h(M[0]+step,M[1]) - h(M[0],M[1])) / step
numerical_J[:,1] = (h(M[0],M[1]+step) - h(M[0],M[1])) / step

        
        
theoreical_J = np.zeros([2,2])
theoreical_J[0,0] = -(2*r+1)/r
theoreical_J[0,1] = 1/r/theta
theoreical_J[1,0] = 2*(r+1)/theta
theoreical_J[1,1] = -1/theta**2

print('check J')
print( (theoreical_J - numerical_J) / numerical_J)

check h: 0.0 0.0
check J
[[-6.61079206e-09 -8.27403642e-08]
 [-3.83314485e-08 -8.27403642e-08]]


In [103]:
## check Sigma_tilde 
numerical_Sigma_tilde = theoreical_J.dot(theoreical_Sigma).dot(theoreical_J.T)

Sigma_tilde_11 = (2*r+3)*theta**2/r \
                 + (4*r+5)*theta/Nr/r*c_gamma_1 \
                 + 2*(r+1)/Nr**2/r*c_gamma_2
        
Sigma_tilde_22 = 2*r*(r+1) \
                 + 4*r*(r+1)/theta/Nr*c_gamma_1 \
                 + 2*r*(r+1)/Nr**2/theta**2*c_gamma_2

print('check Sigma_tilde_11',Sigma_tilde_11 - numerical_Sigma_tilde[0,0])
print('check Sigma_tilde_22',Sigma_tilde_22 - numerical_Sigma_tilde[1,1])


check Sigma_tilde_11 -3.552713678800501e-15
check Sigma_tilde_22 -1.4210854715202004e-14


In [100]:
## check the estimation error 
h_M_sample = np.zeros([n_sample,2])
for i in range(n_sample):
    h_M_sample[i] = h(M_sample[i,0],M_sample[i,1])
    
empirical_h = np.cov(h_M_sample.T)

print('check Sigma_tilde')
print( (empirical_h - numerical_Sigma_tilde) / empirical_h)


/home/martin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in double_scalars
  This is separate from the ipykernel package so we can avoid doing imports until


check Sigma_tilde
[[nan nan]
 [nan nan]]


In [142]:
## check Sigma 
np.random.seed(0)
n_rep = 1000
n_sample = 200

theta = 2
r = 3
Nr = 5

estimate = np.zeros([2,n_rep])
size_factor = np.random.randn(n_sample)*0.1+1

c_gamma_1 = np.mean(1/size_factor)
c_gamma_2 = np.mean(1/size_factor**2)

Sigma_tilde_11 = (2*r+3)*theta**2/r \
                 + (4*r+5)*theta/Nr/r*c_gamma_1 \
                 + 2*(r+1)/Nr**2/r*c_gamma_2
        
Sigma_tilde_22 = 2*r*(r+1) \
                 + 4*r*(r+1)/theta/Nr*c_gamma_1 \
                 + 2*r*(r+1)/Nr**2/theta**2*c_gamma_2
        
        

for i_rep in range(n_rep):
    
    x_sample = np.random.gamma(r,theta,n_sample)
    
    y_sample = np.random.poisson(Nr*size_factor*x_sample)
    M_sample = np.zeros([n_sample,2])
    M_sample[:,0] = y_sample/Nr/size_factor
    M_sample[:,1] = (y_sample**2-y_sample)/Nr**2/size_factor**2
    M_estimate = M_sample.mean(axis=0)
    estimate[:,i_rep] = h(M_estimate[0],M_estimate[1])
    


print('check error theta',((Sigma_tilde_11/n_sample) - np.mean((theta - estimate[0])**2))/ (Sigma_tilde_11/n_sample))
print('check error r',((Sigma_tilde_22/n_sample) - np.mean((r - estimate[1])**2)) / (Sigma_tilde_22/n_sample))


check error theta 0.02682167971413321
check error r -0.0687809116823012


In [148]:
sd.M_to_gamma(np.repeat(M,2).reshape([4,2]))

array([[2., 2.],
       [3., 3.]])

In [146]:
np.repeat(M,2).reshape([4,2])

array([[   6.,    6.],
       [  48.,   48.],
       [ 480.,  480.],
       [5760., 5760.]])